# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
from matplotlib.patches import Rectangle
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from ipywidgets import interact, widgets
import os
from scipy.interpolate import interp1d, griddata
from scipy.signal import savgol_filter
from scipy.interpolate import make_interp_spline
from functions.Hilfsfunktionen_plot import (  plot_polar_subplot,
    plot_component,
    add_break_marks,
    detect_outliers_iqr,
    prepare_plot_data,
    add_region,
    smooth_line,
    prepare_interpolation_data,
    plot_component_with_sem)

# Config

In [2]:
config = {}

# ROMY coordinates
config['ROMY_lon'] = 11.275501
config['ROMY_lat'] = 48.162941

# path for figures to store
config['outpath_figs'] = "C:/Bachelorarbeit/figures/Geschwindigkeit/ROMYRLAS/"
# path for output data
config['outpath_data'] = "C:/Bachelorarbeit/data/waveformsROMYRLAS/"

In [3]:
outpath = config['outpath_figs']
df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")
outpath_FUR ="C:/Bachelorarbeit/figures/Geschwindigkeit/FURWET/"
font_title=17
font_ax=13
font_leg=11

In [25]:
def plot_component(ax, df, title):
    ratio_cols = [col for col in df.columns if col.startswith("ratio_")]
    frequencies = [float(col.split("_")[1][:-2]) for col in ratio_cols]

    # Plot nach Tiefenbereichen
    for (dmin, dmax), label in zip(depth_bins, depth_labels):
        df_subset = df[(df["depth"] >= dmin) & (df["depth"] < dmax)]
        num_events = len(df_subset)
        if df_subset.empty:
            continue

        mean_ratios = df_subset[ratio_cols].mean()
        sem_ratios = df_subset[ratio_cols].sem()
        legend_label = f"depth: {label} (n = {num_events})"

        if dmin == 0 and dmax == 700:
            ax.plot(frequencies, mean_ratios.values, linestyle='--', linewidth=2, label=legend_label)
        else:
            ax.plot(frequencies, mean_ratios.values, linestyle='-', label=legend_label)

        ax.fill_between(frequencies,
                        mean_ratios.values - sem_ratios.values,
                        mean_ratios.values + sem_ratios.values,
                        alpha=0.2)

    # Gesamtmittelwert über alle Tiefen (ohne SEM)
    overall_mean = df[ratio_cols].mean()
    ax.plot(frequencies,
            overall_mean.values,
            color='black',
            linestyle=':',
            linewidth=2,
            label='Overall mean (all depths)')

    ax.set_xlabel("Frequency [Hz]",fontsize=font_ax)
    ax.set_ylabel("Mean ratio (FUR/WET)",fontsize=font_ax)
    ax.set_title(title,fontsize=font_title)
    ax.grid(True)
    ax.legend(fontsize=font_leg)

# Plots

## Verhältnis gegen Frequenz

In [31]:
fig, ax = plt.subplots(figsize=(10, 6))

# Komponenten plotten
plot_component_with_sem(ax, df_Z, color='green', label='Vertical component')

# Achsen und Titel
ax.set_xlabel("Frequency [Hz]",fontsize=font_ax)
ax.set_ylabel("Mean ratio (ROMY/RLAS)",fontsize=font_ax)
ax.set_title("Mean ratio per frequency (± SEM)",fontsize=font_title)
ax.grid(True)
ax.legend(loc='best', fontsize=font_leg)

# Layout & speichern
plt.tight_layout()
plt.savefig(outpath + "meanratio_freqency_Z_SEM_ROMYRLAS.png", dpi=300)
plt.close()


In [46]:
fig, ax = plt.subplots(figsize=(10, 6))
df_T = pd.read_csv(outpath_FUR + "spektralanalyse_ergebnisse_Z.csv")
df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")
df_Tz = pd.read_csv(outpath_FUR + "spektralanalyse_ergebnisse_T.csv")
# Komponenten plotten
plot_component_with_sem(ax, df_T, color='blue', label='FUR/WET (Z)')
plot_component_with_sem(ax, df_Z, color='green', label='ROMY/RLAS (Z)')
plot_component_with_sem(ax, df_Tz, color='orange', label='FUR/WET (T)')
# Achsen und Titel
ax.set_xlabel("Frequency (Hz)",fontsize=font_ax)
#ax.set_xscale('log')
ax.set_ylabel("Mean ratio (ROMY/RLAS)",fontsize=font_ax)
ax.set_title(f"Mean ratio per frequency (± SEM) \n comparison: FUR/WET and ROMY/RLAS",fontsize=font_title)
ax.grid(True)
ax.legend(loc='best', fontsize=font_leg)

# Layout & speichern
plt.tight_layout()
plt.savefig(outpath + "TZZmeanratio_freqency_TandZ_SEM_ROMYRLAS_freqcomparison.png", dpi=300)
plt.close()


# Tiefen


### Verhältnis gegen Frequenz unterteilt in Tiefengruppen

In [26]:
# Konfiguration
depth_bins = [(0, 100), (100, 700)]
depth_labels = ["0–100 km", "100–700 km"]
full_range = (0, 700)

# Subplots erstellen
df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")
fig, axs = plt.subplots(1, 1, figsize=(12, 10), sharex=True)

# Y-Achsenbereiche setzen
##axs[0].set_ylim(0.8, 3.5)
#axs[1].set_ylim(0.8, 1.6)

# Komponenten plotten
plot_component(axs ,df_Z, "Vertical component")
# Formatierung & Speichern
plt.suptitle("Mean ratio (ROMY/RLAS) per frequency divided by depth range (± SEM)", fontsize=font_title)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.savefig(outpath + "meanratio_freqency_depths_ROMYRLAS.png", dpi=300)

plt.close()

## Verhältnis gegen Tiefe


In [34]:
df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")
freq_cols = [col for col in df_Z.columns if col.startswith("ratio_")]

df_Z = df_Z.sort_values("depth")

Z_avg = df_Z[freq_cols].mean(axis=1)

outliers_Z = detect_outliers_iqr(Z_avg)

depth_Z = df_Z["depth"].values

Z_good_depth = depth_Z[~outliers_Z]
Z_good_values = Z_avg[~outliers_Z]

Z_smooth_good = savgol_filter(Z_good_values, window_length=20, polyorder=3)

interp_Z = interp1d(Z_good_depth, Z_smooth_good, kind='linear', bounds_error=False, fill_value="extrapolate")

Z_smooth = interp_Z(depth_Z)

# Tiefenbereiche für Achsenbruch
x1_start, x1_end = 0, 230

range1 = x1_end - x1_start

# Subplots 2x2
fig, ax1_Z = plt.subplots(figsize=(12, 8))

# Z-Komponente
ax1_Z.scatter(depth_Z[~outliers_Z], Z_avg[~outliers_Z], color='green', alpha=0.5,label='Data')
ax1_Z.scatter(depth_Z[outliers_Z], Z_avg[outliers_Z], color='red', alpha=0.7,label='Outliers')
ax1_Z.plot(depth_Z, Z_smooth, color='green', linewidth=2 ,label="Smoothed mean")
ax1_Z.set_xlim(x1_start, x1_end)
ax1_Z.set_ylabel("Mean spectralratio",fontsize=font_ax)
ax1_Z.set_xlabel("Depth (km)",fontsize=font_ax)
ax1_Z.set_title("Vertical component")

ax1_Z.set_ylim(0,17)

fig.suptitle("Spectralratio (ROMY/RLAS) vs. event depths (Mean of all frequencies)",fontsize=font_title) 
# Legenden

ax1_Z.legend(loc="upper right")
plt.savefig(config['outpath_figs'] + "meanratio_vs_depth_ROMYRLAS.png", dpi=300)
plt.close()


C:\Users\bened\anaconda3\envs\ampratio\lib\site-packages\scipy\interpolate\_interpolate.py:479: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
C:\Users\bened\anaconda3\envs\ampratio\lib\site-packages\scipy\interpolate\_interpolate.py:482: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


## Weltkarte

In [35]:
# Regionen definieren
region_bins = [1, 2, 4, 6, 7]
region_labels = ["Europe", "Asia and Philippins", "North- and Southamerica", "Ozeania", "Elsewhere"]
# CSV-Dateien laden

In [6]:
df = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")
# Daten aus den Spalten extrahieren
lats = df['lat'].values
lons = df['lon'].values
magnitudes = df['magnitude'].values
num=len(df)
mapplot = plt.figure(figsize=(14, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAND, facecolor='lightgray')

# Station-Koordinaten
sta_lat = config['ROMY_lat']
sta_lon = config['ROMY_lon']
ax.scatter(sta_lon, sta_lat, color='blue', s=100, marker='*', edgecolor='k', transform=ccrs.PlateCarree(), label='FUR')

ax.legend(loc='upper right',fontsize=font_leg)

sc = ax.scatter(lons, lats, c=magnitudes, cmap='viridis', s=[m**3 for m in magnitudes], alpha=0.7, edgecolor='k', transform=ccrs.PlateCarree())

ax.set_title(f"Earthquake distribution ({num} events)",fontsize=font_title)
ax.gridlines(draw_labels=True)

cbar = plt.colorbar(sc, ax=ax, orientation='vertical', shrink=0.8, pad=0.05)
cbar.set_label("Magnitude",fontsize=font_ax)

# Regionen markieren
add_region(ax, -50, 50, 30, 60, "Region 1 (Europe)", color="red")
add_region(ax, 60, 180, 15, 60, "Region 2 (Asia)", color="blue")
add_region(ax, 100, 180, -10, 15, "Region 2 (Asia)", color="blue")
add_region(ax, -170, -60, 5, 60, "Region 3 (Northamerica)", color="green")
add_region(ax, -120, -50, -80, 5, "Region 3 (Southamerica)", color="green")
add_region(ax, 90, 180, -60, -10, "Region 4 (Ozeania)", color="orange")
add_region(ax, -180, -150, -60, -10, "Region 4 (Ozeania)", color="orange")

# Duplikate in Legende vermeiden
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), loc='lower left',fontsize=font_leg)

mapplot_path = config['outpath_figs'] + "worldmap_earthquke_distribution_ROMYRLAS.png"
mapplot.savefig(mapplot_path, dpi=150, bbox_inches='tight')
print(f" -> Kartenplot gespeichert unter: {mapplot_path}")
plt.close()

 -> Kartenplot gespeichert unter: C:/Bachelorarbeit/figures/Geschwindigkeit/ROMYRLAS/worldmap_earthquke_distribution_ROMYRLAS.png


In [37]:
# Plotdaten extrahieren
df_Z=pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")
ratio_cols_Z, frequencies_Z = prepare_plot_data(df_Z)

mean_ratios_all_Z = df_Z[ratio_cols_Z].mean()
sem_ratios_all_Z = df_Z[ratio_cols_Z].sem()

n_all_Z = len(df_Z)

# Gültige Regionen sammeln (mind. 30 Events in T **oder** Z)
valid_regions = []
for reg, label in zip(region_bins, region_labels):
    n_Z = len(df_Z[df_Z["region"] == reg])
    if  n_Z >= 5:
        valid_regions.append((reg, label, n_Z))

# Subplots vorbereiten
num_plots = len(valid_regions)
fig, axes = plt.subplots(nrows=num_plots, ncols=1, figsize=(10, 4 * num_plots), sharex=True)
if num_plots == 1:
    axes = [axes]

panel_labels = list(string.ascii_lowercase)

for idx, (reg, label, n_Z) in enumerate(valid_regions):
    ax = axes[idx]

    # --- Vertikal-Komponente ---
    if n_Z >= 5:
        df_region_Z = df_Z[df_Z["region"] == reg]
        mean_Z = df_region_Z[ratio_cols_Z].mean()
        sem_Z = df_region_Z[ratio_cols_Z].sem()

        ax.plot(frequencies_Z, mean_Z.values, label=f"Z (n={n_Z})", color='tab:orange')
        ax.fill_between(frequencies_Z,
                        mean_Z.values - sem_Z.values,
                        mean_Z.values + sem_Z.values,
                        alpha=0.2, color='tab:orange')

        # Globaler Mittelwert Z
        ax.plot(frequencies_Z, mean_ratios_all_Z.values, linestyle='--', color='black', label=f"Z global (n={n_all_Z})")
        ax.fill_between(frequencies_Z,
                        mean_ratios_all_Z.values - sem_ratios_all_Z.values,
                        mean_ratios_all_Z.values + sem_ratios_all_Z.values,
                        alpha=0.15, color='gray')

    ax.grid(True)
    ax.set_ylabel("Mean ratio (ROMY/RLAS)",fontsize=font_ax)
    if idx == num_plots - 1:
        ax.set_xlabel("Frequency [Hz]",fontsize=font_ax)

    ax.legend(fontsize=9)
    ax.set_title(f"{panel_labels[idx]}) {label}", loc='left', fontsize=12, fontweight='bold')

# Gesamt-Titel und Layout
fig.suptitle("Mean ratio per frequency for each region (+/-SEM)", fontsize=font_title)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(config['outpath_figs'] + "meanratio_frequency_regions_ROMYRLAS_combined.png", dpi=300)
plt.close()


# Polarplot

## Polarplot Erdbebenverteilung

In [5]:
df = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")
# Werte aus der CSV-Datei extrahieren
theta = np.deg2rad(df['backazimuth'])  # Winkel in Radian
r = df['distance'].values              # Abstand in km
magnitudes = df['magnitude'].values    # Magnitude
num_events = len(df)
# Polarplot erstellen
fig = plt.figure(figsize=(14, 14))
ax = fig.add_subplot(111, polar=True)

sc = ax.scatter(theta, r, c=magnitudes, cmap='viridis', s=[m**3 for m in magnitudes], alpha=0.7)

ax.set_theta_zero_location('N')   # Norden oben
ax.set_theta_direction(-1)        # Uhrzeigersinn
ax.set_title(f"Earthquake distribution relative to FUR ({num_events} events)", va='bottom',fontsize=font_title)
ax.set_rlabel_position(135)
#ax.text(np.deg2rad(132), max(r)*1.37, f"{num_events} events", fontsize=12, 
      #  ha='left', va='top', bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))
# Farbleiste hinzufügen
cbar = plt.colorbar(sc, ax=ax, pad=0.1, shrink = 0.7)
cbar.set_label("Magnitude",fontsize=font_ax)

# Speichern
polarplot_path = config['outpath_figs'] + "polarplot_earthquake_distribution_ROMYRLAS.png"
plt.savefig(polarplot_path, dpi=150, bbox_inches='tight')
print(f" -> Polarplot gespeichert unter: {polarplot_path}")
plt.close()


 -> Polarplot gespeichert unter: C:/Bachelorarbeit/figures/Geschwindigkeit/ROMYRLAS/polarplot_earthquake_distribution_ROMYRLAS.png


## Polarplot mit Verhältnis/Frequenz/Backazimuth

In [39]:
# CSV einlesen
df = pd.read_csv(outpath + "spektralanalyse_ergebnisse_z.csv")

ratio_cols = [col for col in df.columns if col.startswith('ratio_')]
freqs = [float(col.split('_')[1][:-2]) for col in ratio_cols]
ratios = df[ratio_cols].values
backazimuths = np.radians(df['backazimuth'].values)

# Winkel in Radiant umwandeln für polar plot
angles = np.deg2rad(df['backazimuth'].values)

# Erstelle den Plot
fig, ax = plt.subplots(subplot_kw=dict(polar=True), figsize=(8, 8))
#ax = fig.add_subplot(111, polar=True)

# Für jede Frequenz eine Reihe von Punkten plotten
for freq, col in zip(freqs, ratio_cols):
    #radius = freq #(konstant für alle Werte)
    r = np.full_like(angles, freq)
    
    # Werte aus der Spalte, die die Farbe bestimmen
    values = df[col].values
    
    # Scatterplot: Winkel, Radius, Farbe = Werte
    sc = ax.scatter(angles, r, c=values, cmap='viridis', alpha=0.75, s=10)
    
# Farbskala hinzufügen
sc.set_clim(1,20)
cbar = plt.colorbar(sc, pad=0.1, shrink=0.7, extend='both')
cbar.set_label('Spectral ratio (ROMY/RLAS)')

# Achsen-Labels anpassen
#ax.set_rlim(0.001, 0.2)
ax.set_rlabel_position(160)
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)
ax.set_title(f'Spectral ratios as a function of frequency and Backazimuth (Z); n={len(df)})')

# Legende anzeigen
plt.tight_layout()
#plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1)
plt.savefig(config['outpath_figs'] + "polarplot_backazimuth_frequence_z_ROMYRLAS.png", dpi=300,bbox_inches='tight')
plt.close()


## Interpolierter Polarplot mit Verhältnis/Frequenz/Backazimuth

In [11]:
# CSV einlesen
df = pd.read_csv(outpath + "spektralanalyse_ergebnisse_z.csv") 

# Daten vorbereiten
ratio_cols = [col for col in df.columns if col.startswith('ratio_')]
freqs = [float(col.split('_')[1][:-2]) for col in ratio_cols]

# Daten sammeln: Liste von (theta, r, value)
theta_list = []
r_list = []
z_list = []

for freq, col in zip(freqs, ratio_cols):
    theta = np.deg2rad(df['backazimuth'].values)
    r = np.full_like(theta, freq)
    z = df[col].values

    theta_list.extend(theta)
    r_list.extend(r)
    z_list.extend(z)

theta_array = np.array(theta_list)
r_array = np.array(r_list)
z_array = np.array(z_list)

#Periodizität sicherstellen: Daten bei theta + 2π duplizieren
theta_array_extended = np.concatenate([theta_array, theta_array + 2 * np.pi])
r_array_extended = np.concatenate([r_array, r_array])
z_array_extended = np.concatenate([z_array, z_array])

# Gitter erzeugen – endpoint=False vermeidet Dopplung von 0 und 2π
theta_lin = np.linspace(0, 2 * np.pi, 8, endpoint=False)
r_lin = np.linspace(min(freqs), max(freqs), 50)
theta_grid, r_grid = np.meshgrid(theta_lin, r_lin)

# Interpolation durchführen
z_grid = griddata(
    (theta_array_extended, r_array_extended), 
    z_array_extended, 
    (theta_grid, r_grid), 
    method='nearest'
)

# Plot
fig, ax = plt.subplots(subplot_kw=dict(polar=True), figsize=(8, 8))

# Pseudocolor Plot
pcm = ax.pcolormesh(theta_grid, r_grid, z_grid, cmap='viridis', shading='auto')

# Farbskala
pcm.set_clim(1, 20)
cbar = plt.colorbar(pcm, pad=0.1, shrink=0.7, extend='both')
cbar.set_label('Spectral ratio (ROMY/RLAS)')

# Funktion zum Erzeugen eines ausgegrauten Bereichs mit Text
def shaded_sector(ax, theta_start_deg, theta_end_deg, r_inner, r_outer, label):
    theta_start = np.deg2rad(theta_start_deg)
    theta_end = np.deg2rad(theta_end_deg)
    
    # Punkte für den Bereich
    thetas = np.linspace(theta_start, theta_end, 100)
    r1 = np.full_like(thetas, r_inner)
    r2 = np.full_like(thetas, r_outer)
    
    theta_patch = np.concatenate([thetas, thetas[::-1]])
    r_patch = np.concatenate([r1, r2[::-1]])
    
    # Grauer Bereich
    if theta_start_deg ==150:
        ax.fill(theta_patch, r_patch, color='lightgray', alpha=0.85, zorder=10)
    else:
        ax.fill(theta_patch, r_patch, color='lightgray', alpha=0.5, zorder=10)
    
    # Textbeschriftung
    theta_text = (theta_start + theta_end) / 2
    r_text = (r_inner + r_outer) / 2
    ax.text(theta_text, r_text, label, color='black', fontsize=15,
            ha='center', va='center',zorder=11)

# Frequenzbereich anpassen: Hier z. B. von 0.01 bis 0.2 Hz
r_inner = 0
r_outer = 1.9

#ax.set_rlim(0.001, 0.2)
ax.set_rlabel_position(110)
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)
ax.set_title('Spectral ratios as a function of frequency and Backazimuth (Interpolated) (Z)',fontsize=font_title)

plt.tight_layout()
plt.savefig(config['outpath_figs'] + "interpolierter_polarplot_z_ROMYRLAS.png", dpi=300,bbox_inches='tight')
plt.close()


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib import gridspec

# CSV einlesen
df = pd.read_csv(outpath + "spektralanalyse_ergebnisse_z.csv")

# Grunddaten vorbereiten
ratio_cols = [col for col in df.columns if col.startswith('ratio_')]
freqs = [float(col.split('_')[1][:-2]) for col in ratio_cols]
angles = np.deg2rad(df['backazimuth'].values)

# Daten für Interpolation vorbereiten
theta_list, r_list, z_list = [], [], []

for freq, col in zip(freqs, ratio_cols):
    theta = angles
    r = np.full_like(theta, freq)
    z = df[col].values
    theta_list.extend(theta)
    r_list.extend(r)
    z_list.extend(z)

theta_array = np.array(theta_list)
r_array = np.array(r_list)
z_array = np.array(z_list)

# Periodizität sicherstellen
theta_array_ext = np.concatenate([theta_array, theta_array + 2 * np.pi])
r_array_ext = np.concatenate([r_array, r_array])
z_array_ext = np.concatenate([z_array, z_array])

# Nur 8 Segmente (45° Schritte)
theta_lin = np.linspace(0, 2 * np.pi, 8, endpoint=False)
r_lin = np.linspace(min(freqs), max(freqs), 200)
theta_grid, r_grid = np.meshgrid(theta_lin, r_lin)

# Interpolation mit 'nearest'
z_grid = griddata(
    (theta_array_ext, r_array_ext),
    z_array_ext,
    (theta_grid, r_grid),
    method='nearest'
)

# Figure mit benutzerdefiniertem Layout
fig = plt.figure(figsize=(14, 7))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.22)

# ========= Linker Plot: Scatter ========= #
ax1 = fig.add_subplot(gs[0], polar=True)
for freq, col in zip(freqs, ratio_cols):
    r = np.full_like(angles, freq)
    values = df[col].values
    sc = ax1.scatter(angles, r, c=values, cmap='viridis', alpha=0.75, s=10)

ax1.set_rlabel_position(160)
ax1.set_theta_zero_location('N')
ax1.set_theta_direction(-1)
ax1.set_title(r'$\bf{a)}$', loc='left')

# ========= Rechter Plot: Interpoliert ========= #
ax2 = fig.add_subplot(gs[1], polar=True)
pcm = ax2.pcolormesh(theta_grid, r_grid, z_grid, cmap='viridis', shading='auto')
ax1.set_rlim(0.01,1)
ax2.set_rlim(0.01,1)
ax2.set_rlabel_position(110)
ax2.set_theta_zero_location('N')
ax2.set_theta_direction(-1)
ax2.set_title(r'$\bf{b)}$', loc='left')

# ========= Gemeinsame Colorbar ========= #
cax = fig.add_subplot(gs[2])
pcm.set_clim(1, 12)
cbar = fig.colorbar(pcm, cax=cax, extend='both')
cbar.set_label('Spectral ratio (ROMY/RLAS)',fontsize=font_ax)
fig.suptitle('Spectral ratios as a function of frequency and Backazimuth (Z)',fontsize=font_title)
# Speichern
#plt.tight_layout()
plt.savefig(config['outpath_figs'] + "combined_polarplots_z_ROMYRLAS_singlecolorbar_fixed1Hz.png", dpi=300,bbox_inches='tight')
plt.close()


In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib import gridspec

# CSV einlesen
df = pd.read_csv(outpath + "spektralanalyse_ergebnisse_z.csv")

# Grunddaten vorbereiten
ratio_cols = [col for col in df.columns if col.startswith('ratio_')]
freqs = [float(col.split('_')[1][:-2]) for col in ratio_cols]
angles = np.deg2rad(df['backazimuth'].values)

# Daten für Interpolation vorbereiten
theta_list, r_list, z_list = [], [], []

for freq, col in zip(freqs, ratio_cols):
    theta = angles
    r = np.full_like(theta, freq)
    z = df[col].values
    theta_list.extend(theta)
    r_list.extend(r)
    z_list.extend(z)

theta_array = np.array(theta_list)
r_array = np.array(r_list)
z_array = np.array(z_list)

# Periodizität sicherstellen
theta_array_ext = np.concatenate([theta_array, theta_array + 2 * np.pi])
r_array_ext = np.concatenate([r_array, r_array])
z_array_ext = np.concatenate([z_array, z_array])

# Nur 8 Segmente (45° Schritte)
theta_lin = np.linspace(0, 2 * np.pi, 8, endpoint=False)
r_lin = np.linspace(min(freqs), max(freqs), 200)
theta_grid, r_grid = np.meshgrid(theta_lin, r_lin)

# Interpolation mit 'nearest'
z_grid = griddata(
    (theta_array_ext, r_array_ext),
    z_array_ext,
    (theta_grid, r_grid),
    method='nearest'
)

# Figure mit benutzerdefiniertem Layout
fig = plt.figure(figsize=(14, 7))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.22)

# ========= Linker Plot: Scatter ========= #
ax1 = fig.add_subplot(gs[0], polar=True)
for freq, col in zip(freqs, ratio_cols):
    r = np.full_like(angles, freq)
    values = df[col].values
    sc = ax1.scatter(angles, r, c=values, cmap='viridis', alpha=0.75, s=10)

ax1.set_rlabel_position(160)
ax1.set_theta_zero_location('N')
ax1.set_theta_direction(-1)
ax1.set_title(r'$\bf{a)}$', loc='left')

# ========= Rechter Plot: Interpoliert ========= #
ax2 = fig.add_subplot(gs[1], polar=True)
pcm = ax2.pcolormesh(theta_grid, r_grid, z_grid, cmap='viridis', shading='auto')
#ax1.set_rlim(0.01,1)
#ax2.set_rlim(0.01,1)
ax2.set_rlabel_position(110)
ax2.set_theta_zero_location('N')
ax2.set_theta_direction(-1)
ax2.set_title(r'$\bf{b)}$', loc='left')

# ========= Gemeinsame Colorbar ========= #
cax = fig.add_subplot(gs[2])
pcm.set_clim(1, 20)
cbar = fig.colorbar(pcm, cax=cax, extend='both')
cbar.set_label('Spectral ratio (ROMY/RLAS)',fontsize=font_ax)
fig.suptitle('Spectral ratios as a function of frequency and Backazimuth (Z)',fontsize=font_title)
# Speichern
#plt.tight_layout()
plt.savefig(config['outpath_figs'] + "combined_polarplots_z_ROMYRLAS_singlecolorbar_fixed.png", dpi=300,bbox_inches='tight')
plt.close()


## Angular dependencies

In [42]:
# Frequenzbänder inkl. "alle Frequenzen"
bands = {
    'Low (0.01–0.5 Hz)': (0.01, 0.5),
    'Mid (0.5–1 Hz)': (0.5, 1),
    'High (1-2 Hz)': (1, 2),
    'All Frequencies': (0.01, 2)
}
band_labels = list(bands.keys())

# Farben (für 4 Subplots)
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:purple']

In [43]:
# CSV einlesen
df = pd.read_csv(outpath + "spektralanalyse_ergebnisse_z.csv")

# Frequenzspalten extrahieren
ratio_cols = [col for col in df.columns if col.startswith('ratio_')]
freqs = np.array([float(col.split('_')[1][:-2]) for col in ratio_cols])
ratios = df[ratio_cols].values
backazimuths = df['backazimuth'].values

# Subplots vorbereiten (2×2)
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18, 10), sharex=True)
axes = axes.flatten()  # 2D → 1D-Liste
panel_labels = ['a)', 'b)', 'c)', 'd)']
i=0
for ax, label, color in zip(axes, band_labels, colors):
    fmin, fmax = bands[label]
    band_indices = np.where((freqs >= fmin) & (freqs < fmax))[0]
    if len(band_indices) == 0:
        continue

    band_ratios = ratios[:, band_indices]
    mean_ratios = np.mean(band_ratios, axis=1)
    std_ratios = np.std(band_ratios, axis=1)

    # Ausreißer mit IQR entfernen
    Q1 = np.percentile(mean_ratios, 25)
    Q3 = np.percentile(mean_ratios, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    is_outlier = (mean_ratios < lower_bound) | (mean_ratios > upper_bound)
    is_inlier = ~is_outlier

    # Sortieren nach Backazimuth
    sorted_idx = np.argsort(backazimuths)
    sorted_baz = backazimuths[sorted_idx]
    sorted_mean = mean_ratios[sorted_idx]
    sorted_std = std_ratios[sorted_idx]
    sorted_is_inlier = is_inlier[sorted_idx]
    sorted_is_outlier = is_outlier[sorted_idx]

    # Scatterplot Inlier & Outlier
    ax.scatter(sorted_baz[sorted_is_inlier], sorted_mean[sorted_is_inlier],
               color=color, alpha=0.6, s=12, label='Inlier')
    ax.scatter(sorted_baz[sorted_is_outlier], sorted_mean[sorted_is_outlier],
               color='red', marker='x', s=30, label='Outlier')

    # Glättung und Varianz nur mit Inliern
    baz_clean = sorted_baz[sorted_is_inlier]
    mean_clean = sorted_mean[sorted_is_inlier]
    std_clean = sorted_std[sorted_is_inlier]

    if len(baz_clean) > 5:
        baz_interp = np.linspace(baz_clean.min(), baz_clean.max(), 500)
        interp_mean = interp1d(baz_clean, mean_clean, kind='linear')(baz_interp)
        interp_std = interp1d(baz_clean, std_clean, kind='linear')(baz_interp)

        window = min(95, len(baz_interp) // 2 * 2 + 1)
        mean_smooth = savgol_filter(interp_mean, window_length=window, polyorder=3)
        std_smooth = savgol_filter(interp_std, window_length=window, polyorder=3)

        # Mittelwert-Linie
        ax.plot(baz_interp, mean_smooth, color=color, linewidth=2, label='smoothed mean')

        # Varianz-Schatten ±1 Std-Abweichung
        ax.fill_between(baz_interp, mean_smooth - std_smooth, mean_smooth + std_smooth,
                        color=color, alpha=0.3, label='± Standard deviation')

    # Achsen und Styling
    ax.set_title(label)
    ax.set_xlim(0, 360)
    ax.set_ylim(0,16.5)
    ax.grid(True)
    ax.set_xlabel('Backazimuth (°)',fontsize=font_ax)
    ax.set_ylabel('Spectral ratio',fontsize=font_ax)
    ax.legend(loc='upper right',fontsize=font_leg)
    ax.text(0.02, 0.95, panel_labels[i], transform=ax.transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    i+=1
fig.suptitle('Backazimuth vs. spectralratio (ROMY/RLAS) per frequenceband (Vertical component)', fontsize=font_title)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# Speichern
plt.savefig(config['outpath_figs'] + "backazimuth_vs_ratio_standarddeviation_z_ROMYRLAS.png", dpi=300)
plt.close()


In [44]:
# CSV einlesen
df = pd.read_csv(outpath + "spektralanalyse_ergebnisse_z.csv")

# Frequenzspalten extrahieren
ratio_cols = [col for col in df.columns if col.startswith('ratio_')]
freqs = np.array([float(col.split('_')[1][:-2]) for col in ratio_cols])
ratios = df[ratio_cols].values
backazimuths = df['backazimuth'].values

# Subplots vorbereiten (2×2)
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18, 10), sharex=True)
axes = axes.flatten()  # 2D → 1D-Liste
panel_labels = ['a)', 'b)', 'c)', 'd)']
i = 0
for ax, label, color in zip(axes, band_labels, colors):
    fmin, fmax = bands[label]
    band_indices = np.where((freqs >= fmin) & (freqs < fmax))[0]
    if len(band_indices) == 0:
        continue

    band_ratios = ratios[:, band_indices]
    mean_ratios = np.mean(band_ratios, axis=1)
    std_ratios = np.std(band_ratios, axis=1)

    # Sortieren nach Backazimuth
    sorted_idx = np.argsort(backazimuths)
    sorted_baz = backazimuths[sorted_idx]
    sorted_mean = mean_ratios[sorted_idx]
    sorted_std = std_ratios[sorted_idx]

    # Scatterplot aller Datenpunkte
    ax.scatter(sorted_baz, sorted_mean, color=color, alpha=0.6, s=12, label='Data')

    # Glättung und Varianz auf allen Datenpunkten
    if len(sorted_baz) > 5:
        baz_interp = np.linspace(sorted_baz.min(), sorted_baz.max(), 500)
        interp_mean = interp1d(sorted_baz, sorted_mean, kind='linear')(baz_interp)
        interp_std = interp1d(sorted_baz, sorted_std, kind='linear')(baz_interp)

        window = min(95, len(baz_interp) // 2 * 2 + 1)
        mean_smooth = savgol_filter(interp_mean, window_length=window, polyorder=3)
        std_smooth = savgol_filter(interp_std, window_length=window, polyorder=3)

        # Mittelwert-Linie
        ax.plot(baz_interp, mean_smooth, color=color, linewidth=2, label='smoothed mean')

        # Varianz-Schatten ±1 Std-Abweichung
        ax.fill_between(baz_interp, mean_smooth - std_smooth, mean_smooth + std_smooth,
                        color=color, alpha=0.3, label='± Standard deviation')

    # Achsen und Styling
    ax.set_title(label)
    ax.set_xlim(0, 360)
    ax.set_ylim(0,16.5)
    ax.grid(True)
    ax.set_xlabel('Backazimuth (°)',fontsize=font_ax)
    ax.set_ylabel('Spectral ratio',fontsize=font_ax)
    ax.legend(loc='upper right',fontsize=font_leg)
    ax.text(0.02, 0.95, panel_labels[i], transform=ax.transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
    i += 1

fig.suptitle('Backazimuth vs. spectralratio (ROMY/RLAS) per frequenceband (Vertical component)', fontsize=font_title)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# Speichern
plt.savefig(config['outpath_figs'] + "backazimuth_vs_ratio_standarddeviation_z_ROMYRLASnooutliers.png", dpi=300)
plt.close()
